## 1.Dictionary

In [1]:
with open('../Irregular Words/irregular_words.txt', 'r') as dict_file:
    dictionary = {}
    for count, line in enumerate(dict_file, start=1):
        splited = line.strip().split(" ")
        if len(splited) == 2:
            key, value = splited
            dictionary[key] = value

print(f"number of lines {count}")

number of lines 1309


In [2]:
print(dictionary["tokyo"])
print(dictionary["voa"])
print(dictionary["sos"])
print(dictionary["zahra"])

توکیو
وی‌او‌ای
سس
زهرا


## 2.Begining

In [3]:
with open('../Rules/beginning.txt', 'r') as beg_file:
    beginning = {}
    for count, line in enumerate(beg_file, start=1):
        splited = line.strip().split(" ")
        key = splited[0]
        values = splited[1:]
        beginning[key] = values

print(f"number of lines {count}")

number of lines 38


In [4]:
print(beginning["kha"])
print(beginning["sh"])
print(beginning["j"])
print(beginning["ch"])

['خا', 'خوا', 'خ']
['ش']
['ج', 'ژ']
['چ']


## 3.Middle

In [5]:
with open('../Rules/middle.txt', 'r') as mid_file:
    middle = {}
    for count, line in enumerate(mid_file, start=1):
        splited = line.strip().split(" ")
        key = splited[0]
        values = splited[1:]
        middle[key] = values

print(f"number of lines {count}")

number of lines 45


In [6]:
print(middle["sh"])
print(middle["i"])
print(middle["ck"])
print(middle["a"])

['ش']
['ی', 'یع', 'ئی', 'عی']
['ک']
['nothing', 'ع', 'ا', 'اع', 'أ', 'ؤ', 'ؤا', 'ئا', 'ئ']


## 4.Ending

In [7]:
with open('../Rules/ending.txt', 'r') as end_file:
    ending = {}
    for count, line in enumerate(end_file, start=1):
        splited = line.strip().split(" ")
        key = splited[0]
        values = splited[1:]
        ending[key] = values

print(f"number of lines {count}")

number of lines 48


In [8]:
print(ending["sh"])
print(ending["i"])
print(ending["ck"])
print(ending["a"])

['ش']
['ی', 'یع', 'یی']
['ک']
['ا', 'اع', 'nothing']


## 5.Abbreviations

In [9]:
with open('../Rules/chars.txt', 'r') as abbreviation_words:
    abbr = {}
    for count, line in enumerate(abbreviation_words, start=1):
        splited = line.strip().split(" ")
        key = splited[0]
        values = splited[1:]
        abbr[key] = values

print(f"number of lines {count}")

number of lines 62


## 6.WordFreq

In [10]:
with open('../Frequent Persian Words/persian_frequent_words.txt', 'r') as freq_file:
    word_freq = {}
    min_freq = 3
    count = 0
    for count, line in enumerate(freq_file, start=1):
        word, freq = line.strip().split("\t")
        freq = int(freq)
        if freq < min_freq:
            break
        if word not in word_freq:
            word_freq[word] = freq

print(f"number of lines {count}")

number of lines 137255


In [11]:
print(word_freq["یک"])
print(word_freq["جمعیت"])

84193
24228


# Functions

In [12]:
import itertools
import copy

def f2p_word_internal(word, original_word):
    persian = []
    for i, letter in enumerate(word):
        if i == 0:
            converter = beginning
        elif i == len(word) - 1:
            converter = ending
        else:
            converter = middle
        conversions = converter.get(letter)
        if conversions is None:
            return [(original_word, 0.0)]
        conversions = ['' if i == 'nothing' else i for i in conversions]
        persian.append(conversions)

    alternatives = itertools.product(*persian)
    alternatives = [''.join(i) for i in alternatives]
    
    unicode_points = [1607, 8204]
    reconstructed_string = ''.join(chr(point) for point in unicode_points)
    for i in range(len(alternatives)):
        if '@' in alternatives[i]:
            alternatives[i] = alternatives[i].replace('@', reconstructed_string)

    alternatives = [(i, word_freq[i]) if i in word_freq else (i, 0) for i in alternatives]
    
    if len(alternatives) > 0:
        return [max(alternatives, key=lambda x: x[1])]
    else:
        return [(word, 1.0)]

def f2p_word_internal_abbr(original_word, abbr):
    persian = []

    abbr_deepcopy = copy.deepcopy(abbr)
    
    for letter in original_word:
        conversions = abbr_deepcopy.get(letter)
        if conversions is None:
            return [(original_word, 0.0)]
        
        conversion_plus_half_space = [conversion + chr(8204) for conversion in conversions]
        conversions += conversion_plus_half_space
        conversions = ['' if i == 'nothing' else i for i in conversions]
        persian.append(conversions)

    alternatives = itertools.product(*persian)
    alternatives = [''.join(i) for i in alternatives]
    alternatives = [(i, word_freq[i]) if i in word_freq else (i, 0) for i in alternatives]
    
    if len(alternatives) > 0:
        return [max(alternatives, key=lambda x: x[1])]
    else:
        return [(original_word, 1.0)]

In [13]:
def variations(word):
    """Create variations of the word based on letter combinations like oo, sh, etc."""

    if word == 'a':
        return [['A']]
    elif len(word) == 1:
        return [[word[0]]]
    elif word == 'aa':
        return [['A']]
    elif word == 'ee':
        return [['i']]
    elif word == 'iy':
        return [['i']]
    elif word == 'ei':
        return [['ei']]
    elif word in ['oo', 'ou']:
        return [['u']]
    elif word == 'kha':
        return [['kha'], ['kh', 'a']]
    elif word in ['kh', 'gh', 'ch', 'sh', 'zh', 'ck']:
        return [[word]]
    elif word in ["'ee", "'ei"]:
        return [["'i"]]
    elif word in ["'oo", "'ou"]:
        return [["'u"]]
    elif word in ["a'", "e'", "o'", "i'", "u'", "A'"]:
        return [[word[0] + "'"]]
    elif word in ["'a", "'e", "'o", "'i", "'u", "'A"]:
        return [["'" + word[1]]]
    elif len(word) == 2 and word[0] == word[1]:
        return [[word[0]]]

    if word[:2] == 'aa':
        return [['A'] + i for i in variations(word[2:])] + \
               [['a', 'a'] + i for i in variations(word[2:])]
    elif word[:2] in ['ee', 'iy']:
        return [['i'] + i for i in variations(word[2:])]
    elif word[:2] in ['oo', 'ou']:
        return [['u'] + i for i in variations(word[2:])]
    elif word[:3] == 'kha':
        return \
            [['kha'] + i for i in variations(word[3:])] + \
            [['kh', 'a'] + i for i in variations(word[3:])] + \
            [['k', 'h', 'a'] + i for i in variations(word[3:])]
    elif word[:2] in ['kh', 'gh', 'ch', 'sh', 'zh', 'ck']:
        return \
            [[word[:2]] + i for i in variations(word[2:])] 
           # [[word[0]] + i for i in variations(word[1:])]
    elif word[:2] in ["a'", "e'", "o'", "i'", "u'", "A'"]:
        return [[word[:2]] + i for i in variations(word[2:])]
    elif word[:3] in ["'ee", "'ei"]:
        return [["'i"] + i for i in variations(word[3:])]
    elif word[:3] in ["'oo", "'ou"]:
        return [["'u"] + i for i in variations(word[3:])]
    elif word[:2] in ["'a", "'e", "'o", "'i", "'u", "'A"]:
        return [[word[:2]] + i for i in variations(word[2:])]
    elif len(word) >= 2 and word[0] == word[1]:
        return [[word[0]] + i for i in variations(word[2:])]
    else:
        return [[word[0]] + i for i in variations(word[1:])]

    

In [14]:
def f2p_word(word, max_word_size=30):
    original_word = word
    word = word.lower()  # Convert the word to lowercase

    c = dictionary.get(word)
    if c:
        return c  # Return the word from the dictionary if it exists

    if word == '':
        return []
    elif len(word) > max_word_size:
        return [(original_word, 1.0)]  # Return the original word if it exceeds max_word_size

    results = []
    for w in variations(word):
        results.extend(f2p_word_internal(w, original_word))  # Find variations of the word
    
    if len(original_word) <= 4:
        results.extend(f2p_word_internal_abbr(original_word, abbr))  # Consider abbreviations if the word is short
    
    return max(results, key=lambda x:x[1])[0]  # Return the result with the highest score

In [15]:
import re

def split_text(text):
    # Define regex patterns for the three parts
    first_part_pattern = r'^[^a-zA-Z0-9]*'
    middle_part_pattern = r'[a-zA-Z0-9]+'
    last_part_pattern = r'[^a-zA-Z0-9]*$'
    
    # Match the patterns against the text
    first_part_match = re.match(first_part_pattern, text)
    middle_part_match = re.search(middle_part_pattern, text)
    last_part_match = re.search(last_part_pattern, text[middle_part_match.end():]) if middle_part_match else None
    
    # Extract the parts from the matches
    first_part = first_part_match.group(0) if first_part_match else ''
    middle_part = middle_part_match.group(0) if middle_part_match else ''
    last_part = last_part_match.group(0) if last_part_match else text[middle_part_match.end():] if middle_part_match else text
    
    return [first_part, middle_part, last_part]

def add_prefix_postfix(text, prefix, postfix):
    """
    Adds a prefix and a postfix to the given text.
    
    Parameters:
        text (str): The main text.
        prefix (str): The prefix to add.
        postfix (str): The postfix to add.
        
    Returns:
        str: The text with the prefix and postfix added.
    """
    return prefix + text + postfix

# Mapping of English punctuation and special characters to Persian equivalents
english_to_persian = {
    '!': '!',
    '?': '؟',
    '.': '.',
    ',': '،',
    ';': '؛',
    ':': ':',
    '"': '"',
    "'": "'",
    '(': '(',
    ')': ')',
    '[': '[',
    ']': ']',
    '{': '{',
    '}': '}',
    '/': '/',
    '\\': '\\',
    '@': '@',
    '#': '#',
    '$': '$',
    '%': '%',
    '^': '^',
    '&': '&',
    '*': '*',
    '-': '-',
    '_': '_',
    '+': '+',
    '=': '=',
    '~': '~',
    '`': '`',
    '<': '<',
    '>': '>'
}

In [16]:
def f2p(phrase, max_word_size=30):
    """
    Converts an English phrase to its Persian equivalent, including handling punctuation and special characters.
    
    Parameters:
        phrase (str): The English phrase to convert.
        max_word_size (int): The maximum allowed size for a word to be converted.
        
    Returns:
        str: The Persian equivalent of the phrase.
    """
    # Split the phrase into words
    splitted_phrase = re.split(r'[ -]', phrase)

    word_parts = []
    for word in splitted_phrase:
        word_parts.append(split_text(word))  # Split each word into parts

    # Convert special characters
    for i in range(len(word_parts)):
        for char in word_parts[i][0]:
            word_parts[i][0] = word_parts[i][0].replace(char, english_to_persian[char])
        for char in word_parts[i][2]:
            word_parts[i][2] = word_parts[i][2].replace(char, english_to_persian[char]) 

    results = []
    for word in word_parts:
        results.append(f2p_word(word[1], max_word_size))  # Convert the middle part of each word to Persian

    # Add prefix and postfix to the words
    for i in range(len(results)):
        if results[i] != []:
            results[i] = add_prefix_postfix(results[i], word_parts[i][0], word_parts[i][2])
        else:
            if word_parts[i][0] != '':
                results[i] = word_parts[i][0]
            else:
                results[i] = ' '

    return ' '.join(results)  # Join the converted words into a single phrase

# Tests

In [18]:
input = "Salam! Doste aziz. Chetori?"
excepted_output = "سلام! دوست عزیز. چطوری؟"

output = f2p(input)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: سلام! دوست عزیز. چطوری؟
Excepted output: سلام! دوست عزیز. چطوری؟


In [19]:
input = "ax"
excepted_output = "عکس"

output = f2p(input)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: عکس
Excepted output: عکس


In [20]:
input = "Ba zohoore modelhaye zabani bozorg mitavan dar zamane kam khorooji monaseb daryaft kard."
excepted_output = "با ظهور مدل\u200cهای زبانی بزرگ می توان در زمان کم خروجی مناسب دریافت کرد."

output = f2p(input)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: با ظهور مدلهای زبانی بزرگ میتوان در زمان کم خروجی مناسب دریافت کرد.
Excepted output: با ظهور مدل‌های زبانی بزرگ می توان در زمان کم خروجی مناسب دریافت کرد.


In [21]:
input = "mishe ino farsi koni?"
excepted_output = "میشه اینو فارسی کنی؟"

output = f2p(input)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: میشه اینو فارسی کنی؟
Excepted output: میشه اینو فارسی کنی؟


In [22]:
input = "kheili mamnoon"
excepted_output = "خیلی ممنون"

output = f2p(input)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: خیلی ممنون
Excepted output: خیلی ممنون


In [23]:
text = "Emrooz ye rpg zadam be doshman.!"
excepted_output = "امروز یه آرپیجی زدم به دشمن.!"

output = f2p(text)
print("Output:", output)
print("Excepted output:", excepted_output)

Output: امروز یه آرپی‌جی زدم به دشمن.!
Excepted output: امروز یه آرپیجی زدم به دشمن.!


# part 4

In [32]:
fingilish_sentences = [
    "Emrooz hava besyar delpazir bood.",
    "Madaram ghaza-ye khoshmaze-i baraye sham dorost kardeh ast.",
    "Farda be kuhpeymayi khahim raft.",
    "Ketabe jadidi ke kharideh-am besyar jaleb ast.",
    "Doost daram vaghti hava abri ast be park beravam.",
    "Khaneman ra taze rang kardeh-im.",
    "Har rooz sobh yek fenjan chai minusham.",
    "Gorbehaman asheghe bazi ba kamva ast.",
    "Pedaram hamishe mara be varzesh tashvigh mikonad.",
    "Hafte ayande be didare doostanam miravam.",
    "Yadgiriye zabane jadid mitavanad tajrobe jazabi bashad.",
    "Emrooz ba khanevade be sinema raftim.",
    "Man be golhaye hayateman rasidgi mikonam.",
    "Har shab ghabl az khab kami motalee mikonam.",
    "Emtehan hafte ayande kheyli mohem ast.",
    "Az tamashaye ghorube aftab lezzat mibaram.",
    "Behtarin zaman baraye piaderavi sobh zood ast.",
    "Khaharam naghashihaye zibai mikeshad.",
    "Karhaye honari ra kheyli doost daram.",
    "Baraye rooze tavallode doostam yek hediye aali kharidam.",
    "Man vaghti baran mibare kheyli delkhosh mishavam.",
    "Har hafte ba doostam be kafe miravam.",
    "Taaze yeksal ke piano zadan ro shoroo karde-am.",
    "Khodraye nowi kharidim va kheyli razi hastim.",
    "Har sobh be varzesh sabok mashghool mishavam.",
    "Emshab gharar ast pizza dar khane dorost konim.",
    "Az bazdid az mazare tarikhi lezzat mibaram.",
    "In fasl, bagheman por az golhaye rangarang shodeh ast.",
    "Mamoolan shabha filmhaye mostanad tamasha mikonam.",
    "Dars haye khod ra be ezafeye honar va varzesh barname rizi mikonam."
]


In [33]:
persian_sentences = [
    "امروز هوا بسیار دلپذیر بود.",
    "مادرم غذای خوشمزه‌ای برای شام درست کرده است.",
    "فردا به کوهپیمایی خواهیم رفت.",
    "کتاب جدیدی که خریده‌ام بسیار جالب است.",
    "دوست دارم وقتی هوا ابری است به پارک بروم.",
    "خانه‌مان را تازه رنگ کرده‌ایم.",
    "هر روز صبح یک فنجان چای می‌نوشم.",
    "گربه‌ام عاشق بازی با کاموا است.",
    "پدرم همیشه مرا به ورزش تشویق می‌کند.",
    "هفته آینده به دیدار دوستانم می‌روم.",
    "یادگیری زبان جدید می‌تواند تجربه جذابی باشد.",
    "امروز با خانواده به سینما رفتیم.",
    "من به گل‌های حیاطمان رسیدگی می‌کنم.",
    "هر شب قبل از خواب کمی مطالعه می‌کنم.",
    "امتحان هفته آینده خیلی مهم است.",
    "از تماشای غروب آفتاب لذت می‌برم.",
    "بهترین زمان برای پیاده‌روی صبح زود است.",
    "خواهرم نقاشی‌های زیبایی می‌کشد.",
    "کارهای هنری را خیلی دوست دارم.",
    "برای روز تولد دوستم یک هدیه عالی خریدم.",
    "من وقتی باران می‌بارد خیلی دلخوش می‌شوم.",
    "هر هفته با دوستم به کافه می‌روم.",
    "تازه یک سال است که پیانو زدن را شروع کرده‌ام.",
    "خودروی نوی خریدیم و خیلی راضی هستیم.",
    "هر صبح به ورزش سبک مشغول می‌شوم.",
    "امشب قرار است پیتزا در خانه درست کنیم.",
    "از بازدید از مزارع تاریخی لذت می‌برم.",
    "این فصل، باغمان پر از گل‌های رنگارنگ شده است.",
    "معمولاً شب‌ها فیلم‌های مستند تماشا می‌کنم.",
    "درس‌های خود را به اضافی هنر و ورزش برنامه‌ریزی می‌کنم."
]

In [34]:
for f_sentence, p_sentence in zip(fingilish_sentences, persian_sentences):
    print(f2p(f_sentence))
    print(p_sentence)
    print("#"*30)

امروز هوا بسیار دلپذیر بود.
امروز هوا بسیار دلپذیر بود.
##############################
مادرم غذا یه خوشمزه ای برای شام درست کرده است.
مادرم غذای خوشمزه‌ای برای شام درست کرده است.
##############################
فرد به کوهپیمایی خواهیم رفت.
فردا به کوهپیمایی خواهیم رفت.
##############################
کتاب جدیدی که خریده ام بسیار جالب است.
کتاب جدیدی که خریده‌ام بسیار جالب است.
##############################
دوست دارم وقتی هوا ابری است به پارک بروم.
دوست دارم وقتی هوا ابری است به پارک بروم.
##############################
خانمان را تازه رنگ کرده ایم.
خانه‌مان را تازه رنگ کرده‌ایم.
##############################
هر روز صبح یک فنجان چای مینوشم.
هر روز صبح یک فنجان چای می‌نوشم.
##############################
گربهمن عاشق بازی با کاموا است.
گربه‌ام عاشق بازی با کاموا است.
##############################
پدرم همیشه مرا به ورزش تشویق میکند.
پدرم همیشه مرا به ورزش تشویق می‌کند.
##############################
هفته آینده به دیدار دوستانم میروم.
هفته آینده به دیدار دوستانم می‌روم.
####################